# A Test Mode for `SmartPlug`

## Introduction

Developing the `SmartPlug` class has been fun and I would like to build on it. It's a bit unfortunate that at this point I can't really work on `SmartPlug` when I'm not at home, because I can't test anything I write without access to a FRITZ!Box with the relevant home automation devices. I would like to have a way to be able to test mode when I'm on the road.

For that purpose, I would have to simulate the behavior of the FRITZ! smart plugs. I came up with an idea, but before I can explain it, let's take a brief look at how `SmartPlug` works (code snippet below). 

### Review of the `SmartPlug` class

`SmartPlug` is built on top of the class `HomeAutomationDevice` from the component `fritzconnection.lib.fritzhomeauto` of the `fritzconnection` module. But to be clear: it *does not inherit from* `HomeAutomationDevice` in the ordinary Python sense. Rather, the `SmartPlug` constructor takes an instance of `HomeAutomationDevice`, places it as a private attribute, and the remaining class definition borrows some a few attributes and methods from the `HomeAutomationDevice` instance. Here's a stripped down version of the code:


```python
# A stripped down version of the defition of SmartPlug

from fritzconnection.lib.fritzhomeauto import FritzHomeAutomation

class SmartPlug():

    def __init__(self,fritzdevice,idle_threshold=5):
        # private attribute containing a HomeAutomationDevice() instance
        self.__device = fritzdevice
        # some attributes borrowed from the HomeAutomationDevice() instance
        self.identifier = self.__device.identifier
        self.DeviceName = self.__device.DeviceName
        self.model = f"{self.__device.Manufacturer} {self.__device.ProductName}"
        # additional attribute: threshold for power in idle state (measured in Watts)
        self.idle_threshold = idle_threshold

    # some methods borrowed from the HomeAutomationDevice() instance
    def is_switchable(self):
        return self.__device.is_switchable()
    def get_switch_state(self):
        return self.__device.get_switch_state()
    def set_switch(self,arg):
        return self.__device.set_switch(arg)
    def get_basic_device_stats(self):
        return self.__device.get_basic_device_stats()
    
    # some customized methods
    def get_power_stats(self):
        """Get the power statistics recorded by the smart plug."""
        power_stats = self.__device.get_basic_device_stats()['power']
        return power_stats
    
    ...
```

### Strategy for simulating FRITZ! smart plugs

The important thing to understand is how `SmartPlug` interacts with FRITZ! smart plugs, because that's the only things that's missing without access to a suitable FRITZ! home network. This interaction mainly takes place *through the borrowed* methods from `HomeAutomationDevice`. Currentely, these are four: `is_switchable`, `get_switch_state` `set_switch`, `get_basic_device_stats`.

If I want to simulated this behavior, I only have to mimic these four methods.

The crucial observation is that the entire network interaction of a `SmartPlug` instance passes through the `HomeAutomationDevice` instance in the `__device` attribute. 

Here's the general idea: Instead of passing an actual `HomeAutomationDevice` to the the `fritzdevice` argument of the `SmartPlug` constructor, I can pass anything that *behaves like* such an instance. So let's define that type of thing. 

In the following I will define a class `HomeAutomationDeviceSimulator` that has mimcs all the relevant attributes and methods of `HomeAutomationDevice`. Since I'm currently only interested in smart plugs and their power measurements, I'll keep it simple and ignore all other aspects. However, in the future, I might also want to work with thermostats and radiator controls. This would require further work.

## Building the class `HomeAutomationDeviceSimulator` 

Let's start with the attributes. The current implementation of `SmartPlug` makes use of four `HomeAutomationDevice` attributes: `DeviceName`, `Manufacturer`, `ProductName`, and `identifier`. The latter is not really that important, it holds a 12 digit integer (split into a 5+7 digits) which uniquely identifies the hardware (like a MAC address). 

In [2]:
class HomeAutomationDeviceSimulator:

    def __init__(self, name="Smart Plug Simulator"):
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Home Auto Simulator"
        self.identifier = "12345 1234567"

As for the methods, I currently need to mimic the four `HomeAutomationDevice` methods `is_switchable`, `get_switch_state` `set_switch`, `get_basic_device_stats`. For now I'll add placeholders.

In [3]:
class HomeAutomationDeviceSimulator:

    def __init__(self, name="Smart Plug Simulator"):
        # attributes explicitly used by SmartPlug
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Home Auto Simulator"
        self.identifier = "12345 1234567"
    
    def is_switchable(self):
        pass

    def get_switch_state(self):
        pass
    
    def set_switch(self,arg):
        pass
    
    def get_basic_device_stats(self):
        pass


While implementing the methods, we will need to add more attributes that mimic corresponding attributes of `HomeAutomationDevice` instances.

### The easy part: simulating power switching

Let's start building the methods. Since I'm currently only interested in smart plugs, I will simply have `is_switchable` return `True`.

In [4]:
class HomeAutomationDeviceSimulator:

    def __init__(self, name="Smart Plug Simulator"):
        # attributes explicitly used by SmartPlug
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Smart Plug Simulator"
        self.identifier = "12345 1234567"
    
    def is_switchable(self):
        # Assuming that all devices are smart plugs...
        return True

    def get_switch_state(self):
        pass
    
    def set_switch(self,arg):
        pass
    
    def get_basic_device_stats(self):
        pass


For the mehtods `get_switch_state` and `set_switch` we we add an additional attribute `_switch_state`. As you can see, I've chosen to make it a private attribute. There's no real reason, but I find it helpful to distinguish the *implicitly used* attributes from the *explicitly used* ones.

In [5]:
class HomeAutomationDeviceSimulator:

    def __init__(self, name="Smart Plug Simulator"):
        # attributes explicitly used by SmartPlug
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Smart Plug Simulator"
        self.identifier = "12345 1234567"
        # attributes implicitly used by SmartPlug 
        # by way of the methods below
        self._switch_state == True
    
    def is_switchable(self):
        """Checks whether a switchable device is simulated."""
        # Assuming that all devices are smart plugs...
        return True

    def get_switch_state(self):
        """Check whether the simulated device's power switch 
        is on or off."""
        return self.__switch_state
    
    def set_switch(self,target_state:bool):
        """Sets the power switch state of the simulated device."""
        self._switch_state = target_state
    
    def get_basic_device_stats(self):
        pass


### The hard part: simulating power measurements

The real challenge is modeling the real world behavior of the method `get_basic_device_stats`. Here's a first approximation of what happens when you call it in a FRITZ! home automation system:
* Suppose that `homeautodevice` is some instance of `HomeAutomationDevice` that corresponds to some physical home automation device.
* When `hometautohomeautodevice.get_basic_device_stats()` is called, a series of processes in the `fritzconnection` module is triggered, to the effeft that a message is prepared and transmitted the FRITZ!Box router using some protocol whose specifics shall not concern us. 
* The message instructs the FRITZ!Box to forward a request to the phyisical home automation device to return the currently available device statistics, presumably stored in internal memory of the device.
* The device sends this information to the FRITZ!Box which sends it back to our computer and `fritzconnection` parses the received information as a dictionary which is eventually returned by `hometautohomeautodevice.get_basic_device_stats()`.

Here's some code that makes this happen using my `sb4dfritz` package as a short cut. The return values for my setupt are indicates in the comments after `# ->`.

In [13]:
from sb4dfritz import FritzBoxSession

# Connect to Fritzbox
fritzbox = FritzBoxSession()
# Get a list of home automation devices
home_auto_devices = fritzbox.home_auto.get_homeautomation_devices()
# Select one that happens to be a smart plug in my case
homeautodevice = home_auto_devices[3]
print(type(homeautodevice))
    # -> <class 'fritzconnection.lib.fritzhomeauto.HomeAutomationDevice'>
# Call the get_basic_device_stats method
device_stats = homeautodevice.get_basic_device_stats()

<class 'fritzconnection.lib.fritzhomeauto.HomeAutomationDevice'>


Let's take a look at the return value `device_stats`. 

In [ ]:
# Inspective device_stats -> it's a dictionary
print(type(device_stats)) 
    # -> <class 'dict'>
print(device_stats.keys()) 
    # -> dict_keys(['temperature', 'voltage', 'power', 'energy'])

# Inspecting value of device_stats -> again, dictionaries
# They all have the same struture, let's look at one
print(type(device_stats['power'])) 
    # -> <class 'dict'>
print(device_stats['power'].keys()) 
    # -> dict_keys(['count', 'grid', 'datatime', 'data'])

# Inspecting the second level values of device_stats
print(type(device_stats['power']['count']))
    # -> <class 'int'>
print(type(device_stats['power']['grid']))
    # -> <class 'int'>
print(type(device_stats['power']['datatime']))
    # -> <class 'datetime.datetime'>
print(type(device_stats['power']['data']))
    # -> <class 'list'>

As mentioned, we get a nested dictionary, and one with two levels, to be precise. The top level keys indicate the categories of measurements the device performs. In the case of FRITZ! smart plugs, these are temperature, voltage, power, and energy. All the level two dictionaries turn out to have the same structure and contain four key-value pairs. The values have the following interpretation:
* `device_stats[category]['count']`: An integer measuring the number of stored measurements (60 for smart pluge).
* `device_stats[category]['grid']`: An integer indicating the time between measurements in seconds (10 for smartplugs).
* `device_stats[category]['datatime']`: A timestamp of the form `datetime.datetime(YYYY, MM, DD, HH, MM, SS)` indicating the time of the latest measurement *trunctated to second precision(!!!)*.
* `device_stats[category]['data']`: A list of integers recording the measured values. The first entry corresponds to the latest measurement. The units are somewhat odd. For example, for power a value of 100 indicates 1 Watt.

From the values indicated for smart plugs, we see that the devices provide one hour (= 360 * 10 = 3600 seconds) worth of measurements taken every 10 seconds - so it appears, at least. I'll get back to this point and also to the *"truncated to second precision"* part in `datatime`. This will become relevant when we discuss to what extent `get_homeautomation_devices` returns real time information - or rather to what extent FRITZ! smart plugs provide real time information. Spoiler: They do not! There can be considerable latency for a single request. 

## Understanding power measurements of FRITZ! smart plugs

I've already mentioned that one has to expect considerable latency in the information returned from `get_homeautomation_devices`. I'm not entirely sure how exactly the latency comes together - obviously, this is not something you'll find in the documentation of devices that are advertised as providing "monitoring in real time". 

### Sources of latency
To the best of my knowledge, there are at least three sources of latency:

**Network latency:** 
The response time between calling `get_homeautomation_devices` and receiving the measurement data averages at about 0.8 seconds. I've yet to whitness a response time below 0.7 seconds and in 99% of attempts, the response time is below 1.1 seconds.

**Request-Measurement asynchronicity:** 
So, we know that power measurements are taken every 10 seconds and the measurement time is stored with only second precision. To make things worse, it appears that the measurement cycle is not synchronized to the time stamp. The measurements might have been taken up to (not including) 1 second after the time stamp. All in all, if our request happens to reach the smart plug just befor a measurement is taken, we have to expect a latency contribution of up to 11 seconds added to the network latency. Altogether we're up to about 12 seconds of expectable latency, and experiments confirm that this is not too far of. In fact, latency up 13 seconds is not uncommon. That's already quite far from what I would consider real time...

**Sleep mode:** 
But it doesn't even end with the 12-13 seconds. Let me show you an example of `device_stats[category]['data']`. Bare in mind that the smart plug hasn't received any network requests in some time.

In [ ]:
print(device_stats['power']['data'][:60])
    # -> [36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 
    #     329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 
    #     329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 
    #     329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 
    #     20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577]

[36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577]


Notice something? Oddly, the values only change every 12 cycles, aka 10 * 12 = 120 seconds, aka 2 minutes. Now watch what happens, if several requests are sent in a row: 

In [ ]:
from time import sleep

device_stats = homeautodevice.get_basic_device_stats()
sleep(10)
device_stats = homeautodevice.get_basic_device_stats()
sleep(10)
device_stats = homeautodevice.get_basic_device_stats()
print(device_stats['power']['data'][:120])
    # -> [336, 336, 336, 336, 30190, 329, 329, 329, 329, 329, 329, 329, 
    #     329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 
    #     329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 
    #     329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 8554, 8554, 
    #     8554, 8554, 8554, 8554, 8554, 8554, 336, 329, 329, 336, 336, 336, 
    #     329, 329, 336, 336, 43866, 43866, 43866, 43866, 43866, 43866, 43866, 43866, 
    #     36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 36950, 
    #     329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 
    #     329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 
    #     329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329]

[336, 336, 336, 336, 30190, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329]


The seventh line in this example is the same as the first line in the previous example. In fact, they described the same measurements. Pay attention to what happens after this, that is, in the first six lines. Most of the time, the value indicates 3.29 W which is the idle power consumption of the connected appliance. However, there are instances where the values change with every cycle (e.g. in the first line). 

This suggests that the sensory components of the smart plug go into sleep mode after longer periods without incoming network requests. During sleep mode, measurements are only updated every 2 minutes. Once a request arrives, the device wakes up and takes power measurements every 10 seconds. What I haven't been able to understand is how long exactly the device stays awake. I'm suspecting that it is a rather short time span of at most 10 minutes, possibly even less.

Now that does not seem like not entirely unreasonable behavior from the perspective of energy efficiency. In most scenarios, the smart plug itself is arguable meant to save energy in the long run rather than consume energy for measurements. So it would make sense to design the smart plugs in such a way that the consume minimal amounts of energy. However, I can't imagine that the sensors would require considerable amounts of energy, even without the alleged sleep mode. Anyhow, it is what it is.

**To summarize:** All in all, it is possible that the latest measurement was taken up to ~133 seconds *before* the power data was return by `get_basic_device_stats`. So much for "real time"...

In all fairness, I cannot say that this is entirely caused by the FRITZ! devices. After all, something in the implementation of `fritzconnection` might cause some amount of latency. But judging from experience with the official FRITZ! Smart Home app and the FRITZ!Box web interface, the latter seems unlikely. Sorry, AVM guys...

### Cycle resets

Another bit of curiosity, which is possibly related to the sleep mode, is that the smart plugs measurement cycle does not appear to be fixed. Let's take a look at the time stamp of my last request, and then make another request.

In [ ]:
print(device_stats['power']['datatime'])
    # -> 2025-07-06 16:22:58
device_stats = homeautodevice.get_basic_device_stats()
print(device_stats['power']['datatime'])
    # -> 2025-07-06 16:54:34

If the cycle was fixed, the second values should have agreed in the last digit. Well, they don't. At the moment, I don't have a good explanation for this. I don't even have a guess that's worth mentioning. However, what I can say is that the measurement cycle does not reset if requests are made regularly. 

### How to optimize response time latency?

Since I'm rather far astray anyway, let me mention why I started working on `sb4dfritz` in the first place. The appliance connected to my example smart plug is an espresso machine which has irregular heating cycles to keep the water in its boiler at temperature. I'm using the smart plug as a remote switch. Although I've been assured by a certified service technician that the machine does not suffer if it is switched off during a heating cycle, I much prefer to switch it off in an idle cycle. 

When I first came across `fritzconnection`, I though I could achieve this in a few lines of code: simply ask for the current power consumption, switch of if the power consumption indicated idle state, and repeat this cycle until idle state is reported. Having written these few lines of code, it started daunting on me that the smart plug didn't really return "real time information".

After some more experimentation, I had understood the behavior explained above. So I went ahead and started looking for a way to schedule requests to the smart to guarantee low latency. I eventually settled on the approach taken in `sb4dfritz` to set up a request cycle parallel to the measuring cycle and to adjust the offset to get the latency as low as possible. Even this is not entirely straight forward due to network latency fluctuation. There is a range of borderline offsets which will fluctuate between returning results with  best possible or worst possible latency.

## Back to simulating power measurements

Let's get back to the actual plan to simulate the behavior of FRITZ! smart plugs. This is what we have so far:

In [30]:
class HomeAutomationDeviceSimulator:

    def __init__(self, name="Smart Plug Simulator"):
        # attributes explicitly used by SmartPlug
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Smart Plug Simulator"
        self.identifier = "12345 1234567"
        # attributes implicitly used by SmartPlug 
        # by way of the methods below
        self._switch_state == True
    
    def is_switchable(self):
        """Checks whether a switchable device is simulated."""
        # Assuming that all devices are smart plugs...
        return True

    def get_switch_state(self):
        """Check whether the simulated device's power switch 
        is on or off."""
        return self.__switch_state
    
    def set_switch(self,target_state:bool):
        """Sets the power switch state of the simulated device."""
        self._switch_state = target_state
    
    def get_basic_device_stats(self):
        pass

Thanks to the lengthy detour leading up to this we now have a better understanding of what we have to do:
* We understand the data structure that our fake `get_basic_device_stats` has to return.
* We have learned that the most complicated part will be modeling all the odd timing issues.


### Another class: `MeasurementSimulator`
So, how to approach the implementation, then? I chose to create another class `MeasurementSimulator` which plays the role of the physical smart switch. It will hold fake device stats in its internal memory, send this information on request, have measurement cycles, a sleep mode, etc.

Further, in the `HomeAutomationDeviceSimulator` we add an attribute `sensor` which holds a dedicated `MeasurementSimulator` instance. The `get_basic_device_stats` method in the former will trigger a `send_basic_device_stats` in the latter. Here's a first template:

In [ ]:
class MeasurementSimulator():
    
    def __init__(self):
        self.basic_device_stats = None
        self.cycle_base_time = None
        self.sleeping = False
        
    def send_basic_device_stats(self):
        pass

    def reset_cycle_base_time(self):
        pass

    def stay_awake(self,awake_time):
        pass

class HomeAutomationDeviceSimulator:

    def __init__(self, name="Smart Plug Simulator"):
        # attributes explicitly used by SmartPlug
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Smart Plug Simulator"
        self.identifier = "12345 1234567"
        # attributes implicitly used by SmartPlug 
        # by way of the methods below
        self._switch_state == True
        self.sensor:MeasurementSimulator = MeasurementSimulator()
    
    def is_switchable(self):
        """Checks whether a switchable device is simulated."""
        # Assuming that all devices are smart plugs...
        return True

    def get_switch_state(self):
        """Check whether the simulated device's power switch 
        is on or off."""
        return self.__switch_state
    
    def set_switch(self,target_state:bool):
        """Sets the power switch state of the simulated device."""
        self._switch_state = target_state
    
    def get_basic_device_stats(self):
        return self.sensor.send_basic_device_stats()

At this point, I can report some good news: The `HomeAutomationDeviceSimulator` has reached its final form. The missing behavior of the method `get_basic_device_stats` will be implemented in the `MeasurementSimulator` class. For the time being, we will exclusively work on the code for `MeasurementSimulator`.

#### Implementing the measure cycle reset

In [ ]:
from datetime import datetime, timedelta
import random
import threading

class MeasurementSimulator():
    
    def __init__(self):
        self.basic_device_stats = None
        # set cycle base time
        self.cycle_base_time = None
        self.reset_cycle_base_time()
        # keep awake after initialization
        self.stay_awake()

    def send_basic_device_stats(self):
        # wake up the device
        self.stay_awake()
        # return the curent device stats
        return self.basic_device_stats
    
    def reset_cycle_base_time(self):
        # get current time minus one second
        t = datetime.now() + timedelta(seconds=-1)
        # replace the microsecond component with a random value
        t = t.replace(microsecond=random.randint(0,1000000))
        # update the cycle base time
        self.cycle_base_time = t

    def stay_awake(self):
        pass


#### Implementing a sleep mode

Let's quickly recap what happens IRL. When the smart plug is plug in, I imagine it goes into awake mode and stays awake for at least that long. If no network request comes in, it goes to sleep after that time. If a network request comes in while the device is sleeping, it has the same effect as plugging the smart plug it. If a network request arrive within an awake period, the awake period is extended. All of this happens while independently of the rest of the home automation system. 

Implementing this will require a program with multiple parallel threads. I'll be honest, I didn't quite know how to build this myself. But ChatGPT came to my rescue and gave me a nice solution template.

For the moment, I'll work exclusively on the `MeasurementSimulator` code. Here's an implementation of sleep mode and a method to send device stats. However, note that we don't have any device stats, yet! Those will be added later. As for the sleep mode, I will make the following assumptions:
* Sleep mode is activated after 5 minutes (300 seconds) of inactivity.
* If `send_basic_device_stats` is triggered while the device is sleeping, the measure cycle is reset.

In [ ]:
from datetime import datetime, timedelta
import random
import threading

class MeasurementSimulator():
    
    def __init__(self):
        self.basic_device_stats = None
        # set cycle base time
        self.cycle_base_time = None
        self.reset_cycle_base_time()
        # attributes needed for sleep mode
        self.sleeping = False
        self._lock = threading.Lock()
        self._timer = None
        # keep awake after initialization
        self.stay_awake()

    def send_basic_device_stats(self):
        # wake up the device
        self.stay_awake()
        # return the curent device stats
        return self.basic_device_stats
    
    def reset_cycle_base_time(self):
        # get current time minus one second
        t = datetime.now() + timedelta(seconds=-1)
        # replace the microsecond component with a random value
        t = t.replace(microsecond=random.randint(0,1000000))
        # update the cycle base time
        self.cycle_base_time = t

    def _go_to_sleep(self):
        with self._lock:
            self.sleeping = True

    def stay_awake(self, awake_time=300):
        if self.sleeping:
            self.reset_cycle_base_time()
        with self._lock:
            self.sleeping = False
            if self._timer:
                self._timer.cancel()
            self._timer = threading.Timer(awake_time, self._go_to_sleep)
            self._timer.daemon = True
            self._timer.start()


Let's go for a test drive. We'll set the awake time to 5 seconds, print the sleep status every second 20 times and call `stay_awake` on three times, with distance more (resp. less) than five between the first (resp. last) two iterations. This way `sensor` should go to sleep 5 seconds after the first call, but not after the second.

In [81]:
from time import sleep

sensor = MeasurementSimulator()
sensor._go_to_sleep()

for i in range(20):
    if i in [2, 10, 13]:
        sensor.stay_awake(5)
    print(f"{i:2d} - Cycle base time:", sensor.cycle_base_time, " |  Sensor sleepign?", sensor.sleeping)
    sleep(1)


 0 - Cycle base time: 2025-07-06 22:35:50.760031  |  Sensor sleepign? True
 1 - Cycle base time: 2025-07-06 22:35:50.760031  |  Sensor sleepign? True
 2 - Cycle base time: 2025-07-06 22:35:52.218090  |  Sensor sleepign? False
 3 - Cycle base time: 2025-07-06 22:35:52.218090  |  Sensor sleepign? False
 4 - Cycle base time: 2025-07-06 22:35:52.218090  |  Sensor sleepign? False
 5 - Cycle base time: 2025-07-06 22:35:52.218090  |  Sensor sleepign? False
 6 - Cycle base time: 2025-07-06 22:35:52.218090  |  Sensor sleepign? False
 7 - Cycle base time: 2025-07-06 22:35:52.218090  |  Sensor sleepign? False
 8 - Cycle base time: 2025-07-06 22:35:52.218090  |  Sensor sleepign? True
 9 - Cycle base time: 2025-07-06 22:35:52.218090  |  Sensor sleepign? True
10 - Cycle base time: 2025-07-06 22:36:00.847552  |  Sensor sleepign? False
11 - Cycle base time: 2025-07-06 22:36:00.847552  |  Sensor sleepign? False
12 - Cycle base time: 2025-07-06 22:36:00.847552  |  Sensor sleepign? False
13 - Cycle base 

#### A first step toward device stats

In [94]:
from datetime import datetime, timedelta
import random
import threading

class MeasurementSimulator():
    
    def __init__(self):
        self.basic_device_stats = None
        self._generate_basic_device_stats()
        # set cycle base time
        self.cycle_base_time = None
        self.reset_cycle_base_time()
        # attributes needed for sleep mode
        self.sleeping = False
        self._lock = threading.Lock()
        self._timer = None
        # keep awake after initialization
        self.stay_awake()

    # TODO figure out how to add reasonable 'data' values
    def _generate_basic_device_stats(self):
        # designated dictionary keys
        LEVEL_1_KEYS = ['temperature', 'voltage', 'power', 'energy']
        LEVEL_2_KEYS = ['count', 'grid', 'datatime', 'data']
        # designates values for 'count' and 'grid' in order of LEVEL_1_KEYS
        COUNT_VALUES = [96, 360, 360, 12]
        GRID_VALUES = [900, 10, 10, 2678400]
        # initializing the nested dictionary
        level2_template = {key2:None for key2 in LEVEL_2_KEYS}
        device_stats = {key1:level2_template.copy() for key1 in LEVEL_1_KEYS}
        # adding in the values in level 2
        for idx, category in enumerate(LEVEL_1_KEYS):
            device_stats[category]['count'] = COUNT_VALUES[idx]
            device_stats[category]['grid'] = GRID_VALUES[idx]
            device_stats[category]['datatime'] = self._get_latest_measure_timestamp()
            device_stats[category]['data'] = self._generate_measuredata(category)
        # updating the basic_device_stats attribute
        self.basic_device_stats = device_stats
    

    # TODO decide what to do with the timestamp
    def _get_latest_measure_timestamp(self):
        return datetime.now().replace(microsecond=0)
    
    # TODO find a reasonable way to generate data
    def _generate_measuredata(self, category):
        return []

    def send_basic_device_stats(self):
        # wake up the device
        self.stay_awake()
        # return the curent device stats
        return self.basic_device_stats
    
    def reset_cycle_base_time(self):
        # get current time minus one second
        t = datetime.now() + timedelta(seconds=-1)
        # replace the microsecond component with a random value
        t = t.replace(microsecond=random.randint(0,1000000))
        # update the cycle base time
        self.cycle_base_time = t

    def _go_to_sleep(self):
        with self._lock:
            self.sleeping = True

    def stay_awake(self, awake_time=300):
        if self.sleeping:
            self.reset_cycle_base_time()
        with self._lock:
            self.sleeping = False
            if self._timer:
                self._timer.cancel()
            self._timer = threading.Timer(awake_time, self._go_to_sleep)
            self._timer.daemon = True
            self._timer.start()


In [96]:
sensor = MeasurementSimulator()

sensor.send_basic_device_stats()

{'temperature': {'count': 96,
  'grid': 900,
  'datatime': datetime.datetime(2025, 7, 6, 23, 8, 36),
  'data': []},
 'voltage': {'count': 360,
  'grid': 10,
  'datatime': datetime.datetime(2025, 7, 6, 23, 8, 36),
  'data': []},
 'power': {'count': 360,
  'grid': 10,
  'datatime': datetime.datetime(2025, 7, 6, 23, 8, 36),
  'data': []},
 'energy': {'count': 12,
  'grid': 2678400,
  'datatime': datetime.datetime(2025, 7, 6, 23, 8, 36),
  'data': []}}

*<center>(continue here...)</center>*

In [67]:
import datetime 
import random

class MeasurementSimulator():
    
    def __init__(self):
        self._datatime = datetime.datetime.now().replace(microsecond=0)
        self._offset = random.random()
        self.sleeping = True
        self._basic_device_stats = None
        self._generate_basic_device_stats()
    
    # TODO figure out how to add reasonable 'data' values
    def _generate_basic_device_stats(self):
        # designated dictionary keys
        LEVEL_1_KEYS = ['temperature', 'voltage', 'power', 'energy']
        LEVEL_2_KEYS = ['count', 'grid', 'datatime', 'data']
        # designates values for 'count' and 'grid' in order of LEVEL_1_KEYS
        COUNT_VALUES = [96, 360, 360, 12]
        GRID_VALUES = [900, 10, 10, 2678400]
        # initializing the nested dictionary
        level2_template = {key2:None for key2 in LEVEL_2_KEYS}
        device_stats = {key1:level2_template.copy() for key1 in LEVEL_1_KEYS}
        # adding in the values in level 2
        for idx, key in enumerate(LEVEL_1_KEYS):
            device_stats[key]['count'] = COUNT_VALUES[idx]
            device_stats[key]['grid'] = GRID_VALUES[idx]
            device_stats[key]['datatime'] = self._datatime
            device_stats[key]['data'] = [0] * COUNT_VALUES[idx]
        # updating the basic_device_stats attribute
        self._basic_device_stats = device_stats
    
    # TODO flesh this out
    def send_basic_device_stats(self):
        # wake up
        self.set_sleep_state(False)
        # return data
        return self._basic_device_stats

    # TODO This is either useless or it needs to be fancy
    def set_sleep_state(self, sleep_state:bool):
        self.sleeping = sleep_state


class HomeAutomationDeviceSimulator:

    def __init__(self, name="Smart Plug Simulator"):
        # attributes explicitly used by SmartPlug
        self.DeviceName = name 
        self.Manufacturer = "SB4D" 
        self.ProductName = "FRITZ! Smart Plug Simulator"
        self.identifier = "12345 1234567"
        # attributes implicitly used by SmartPlug 
        # by way of the methods below
        self._switch_state == True
        self.sensor:MeasurementSimulator = MeasurementSimulator()
    
    def is_switchable(self):
        """Checks whether a switchable device is simulated."""
        # Assuming that all devices are smart plugs...
        return True

    def get_switch_state(self):
        """Check whether the simulated device's power switch 
        is on or off."""
        return self.__switch_state
    
    def set_switch(self,target_state:bool):
        """Sets the power switch state of the simulated device."""
        self._switch_state = target_state
    
    # TODO implement this
    def get_basic_device_stats(self):
        return self.sensor.send_basic_device_stats()

*<center>(to be continued...)</center>*

# Workbench

Trying things out. Don't expect to understand what I'm doing.

In [69]:
sensor = MeasurementSimulator()

sensor.send_basic_device_stats()

{'temperature': {'count': 96,
  'grid': 900,
  'datatime': datetime.datetime(2025, 7, 6, 18, 50, 40),
  'data': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]},
 'voltage': {'count': 360,
  'grid': 10,
  'datatime': datetime.datetime(2025, 7, 6, 18, 50, 40),
  'data': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
  